In [1]:
import pickle as pkl
import spacy
import csv
import json
from copy import deepcopy
from tqdm import tqdm
import benepar

In [79]:
sm_parser = spacy.load('en_core_web_sm')
berkeley_parser = spacy.load('en_core_web_md')
berkeley_parser.add_pipe("benepar", config={"model": "benepar_en3"})
# stanza_parser = spacy_stanza.load_pipeline('en')
trf_parser = spacy.load("en_core_web_trf")

In [80]:
# Load Parsed Corpus
sm_parser = spacy.load('en_core_web_sm')

with open('tbbt_en_zh.pkl', 'rb') as f_zh:
    with open('tbbt_en_fa.pkl', 'rb') as f_fa:
        zh = pkl.load(f_zh)
        fa = pkl.load(f_fa)
        inter_keys = set(zh.keys()) & set(fa.keys())

with open('parsed_corpus.pkl', 'rb') as f:
    nps = pkl.load(f)
with open('parsed_corpus_all.pkl', 'rb') as f:
    tags = pkl.load(f)

In [81]:
# Merge
def merge_maximum_span(spans):
    spans.sort(key=lambda x: x[1])
    to_pop = []
    for j, (word_0, start_idx_0, end_idx_0) in enumerate(spans):
        for k, (word_1, start_idx_1, end_idx_1) in enumerate(spans):
            if k==j:
                continue
            if (start_idx_1 >= start_idx_0) and (end_idx_1 <= end_idx_0):
                to_pop.append(spans[k])
    for item in to_pop:
        if item in spans:
            spans.remove(item)

    spans.sort(key=lambda x: x[1])

    return spans

In [82]:
# Collect Data
output = []
for epi_key in inter_keys:
    if epi_key != (1,8):
        continue
    for i in range(len(nps[epi_key])):
        all_sentences = []
        all_query_spans = []

        scene = nps[epi_key][i]
        scene_tag = tags[epi_key][i]
        j = 0
        for utt, tag in zip(scene, scene_tag):
            speaker = utt['speaker'].strip().strip("(").strip(")").strip().strip(".").strip().strip(":")
            speaker_tokens = [item.text for item in sm_parser(speaker)]
            if "en_subtitles" not in utt:
                sentence = utt['utterance']
                utt['sm_noun_chunk'] = [(item.text, item.start, item.end) for item in sm_parser(sentence).noun_chunks]
                utt['sm_pron'] = [(item.text,i, i+1) for i, item in enumerate(sm_parser(sentence)) if item.pos_=="PRON"]
                utt['berkeley_noun_chunk'] = [(item.text, item.start, item.end) for item in berkeley_parser(sentence).noun_chunks]
                utt['berkeley_pron'] = [(item.text,i, i+1) for i, item in enumerate(berkeley_parser(sentence)) if item.pos_=="PRON"]
                utt['trf_noun_chunk'] = [(item.text, item.start, item.end) for item in trf_parser(sentence).noun_chunks]
                utt['trf_pron'] = [(item.text,i, i+1) for i, item in enumerate(trf_parser(sentence)) if item.pos_=="PRON"]
            else:
                sentence = " ".join([x.strip().lstrip('-').lstrip().lstrip('.').lstrip() for x in utt['en_subtitles']])
            sentence_token = [item.text for item in sm_parser(sentence)]
            noun_phrase = merge_maximum_span(list(set(utt['sm_noun_chunk']) | set(utt['berkeley_noun_chunk']) | set(utt['trf_noun_chunk'])))
            pron = merge_maximum_span(list(set(utt['sm_pron']) | set(utt['berkeley_pron']) | set(utt['trf_pron'])))
            mention = list(set(noun_phrase)|set(pron))
            mention.sort(key=lambda x: x[1])

            all_sentences.append(speaker_tokens + [":"] + sentence_token)
            for span in mention:
                all_query_spans.append({
                        "sentenceIndex": j,
                        "startToken": span[1] + len(speaker_tokens) + 1,
                        "endToken": span[2] + len(speaker_tokens) + 1
                    })
            j+=1
        output.append({
            "sentences": all_sentences,
            "querySpans": all_query_spans,
            "candidateSpans": all_query_spans,
            "clickSpans": all_query_spans,
            })

with open('sample_annotate_epi_1_8_new.csv', "w", encoding="utf-8") as csv_fh:
        fieldnames = ['json_data']
        writer = csv.DictWriter(csv_fh, fieldnames, lineterminator='\n')
        writer.writeheader()
        for line in output:
            writer.writerow({'json_data': json.dumps(line)})